In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-5d49  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  7 22:00 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '18071996' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if 'parquet' in b.name:
        paths.append(full_path+b.name)

# Building an inverted index

In [8]:
parquetFile = spark.read.parquet(*paths)
#create rdd of (text, id)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Import the inverted index module

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

In [18]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

stemming_dict = {}
stemmer = PorterStemmer()

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, f, doc size)) pairs
      for example: [("Anarchism", (12, 5, 35)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  ##contains the words and how many times it appeared in the given document
  word_dict={}
  num_of_words_per_doc = 0
  for w in tokens:
    if w not in all_stopwords:
        num_of_words_per_doc = num_of_words_per_doc+1
        #if we didn't see this word
        if w not in stemming_dict:
          stemming_dict[w] = stemmer.stem(w)
          word = stemming_dict[w]
          if word not in word_dict:
            word_dict[word] = [id,1]
          else:
            word_dict[word][1] = word_dict[word][1]+1
        else:
            word = stemming_dict[w]
            if word not in word_dict:
              word_dict[word] = [id,1]
            else:
              word_dict[word][1] = word_dict[word][1]+1   
    if len(stemming_dict) >= 30000:
        stemming_dict.clear()
  return [(stemmed_word, (tup[0],tup[1],num_of_words_per_doc)) for stemmed_word,tup in word_dict.items()]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  sorted_list = sorted(unsorted_pl, key=(lambda x: x[0]))
  return sorted_list

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  flattened_rdd = postings.flatMap(lambda x: [(x[0], 1) for (di, num) in x[1]])
  df_rdd = flattened_rdd.reduceByKey(lambda x, y: x + y)
  return df_rdd

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE
  inverted = InvertedIndex()
  grouped_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

  posting_locations_rdd = grouped_rdd.mapValues(list).map(lambda x: inverted.write_a_posting_list(x,"text_inverted_gcp",bucket_name))

  return posting_locations_rdd

In [19]:
# this row takes the rdd doc_text_pairs and run the method word_count for each text and id
text_word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

In [20]:
# transform the text_word_counts into dict. the key is the id, and the value is the doc size
# dict_Id_Size = text_word_counts.map(lambda x: (x[1][0], x[1][2])).reduceByKey(lambda x, y: x).map(lambda x: (x[0], x[1]))
# dict_Id_Size = dict_Id_Size.collectAsMap()

In [21]:
# save_dict_to_pickle(dict_Id_Size, "dict_Id_Size")

In [22]:
#methods of saving instance to a pickle file and load it
def save_dict_to_pickle(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)

def load_dict_from_pickle(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [23]:
#method of uploading pickle file from the bucket
def load_dict_from_pickle_bucket(bucket_name, filename):
    """Load a dictionary from a pickle file stored in a GCS bucket."""
    # Initialize a client
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # Get the blob (file) from the bucket
    blob = bucket.blob(filename)

    # Download the blob's content as bytes
    content = blob.download_as_string()

    # Deserialize the bytes content into a Python object (dictionary in this case)
    loaded_dict = pickle.loads(content)

    return loaded_dict

dict_Id_Size = load_dict_from_pickle_bucket("18071996","files/dict_Id_Size")
print(dict_Id_Size[12])

4064


In [24]:
# create an rdd of (word, (doc_id, tf))
text_word_counts = text_word_counts.map(lambda x: (x[0], (x[1][0], x[1][1])))
# create an rdd of (word, [(doc_id, tf),...]) and returns it sorted
text_postings = text_word_counts.groupByKey().mapValues(reduce_word_counts)

In [25]:
#the text_postings is too big so we had to filtter some
postings_filtered = text_postings.filter(lambda x: len(x[1])>50)

In [26]:
# returns an rdd of word and its df
text_w2df = calculate_df(postings_filtered)

In [22]:
#transforms the rdd to a dict of word as a key, df as a value
# dict_text_w2df = text_w2df.collectAsMap()
# dict_text_w2df = load_dict_from_pickle("dict_text_w2df")

In [23]:
# save_dict_to_pickle(dict_text_w2df, "dict_text_w2df")

In [27]:
#returns an rdd representing where each posting list is written
text_posting_locs = partition_postings_and_write(postings_filtered).collect()
# text_posting_locs = load_dict_from_pickle("text_posting_locs")

In [25]:
# save_dict_to_pickle(text_posting_locs, "text_posting_locs")

In [28]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='text_inverted_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
#creates an rdd of (id, anchor text)
pages_links = parquetFile.select("id", "anchor_text").rdd

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  vertices = pages.flatMap(lambda row: [(row.id,"")] + [(anchor.id,"") for anchor in row.anchor_text]).distinct()
  edges = pages.flatMap(lambda row: [(row.id, anchor.id) for anchor in row.anchor_text]).distinct()
  vertices.persist()
  edges.persist()
  return edges, vertices

**PAGE RANK**

In [ ]:
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pagerank_df = pr_results.vertices
pagerank_dict = pagerank_df.rdd.map(lambda x: (x['id'], x['pagerank'])).collectAsMap()

In [51]:
pagerank_dict = load_dict_from_pickle("pagerank_dict")

In [29]:
# Create inverted index instance
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs

In [ ]:
# Add the token - df dictionary to the inverted index
dict_Id_Size = load_dict_from_pickle_bucket("18071996","files/dict_Id_Size")
inverted_text.idSize = dict_Id_Size
dict_text_w2df = load_dict_from_pickle_bucket("18071996","files/dict_text_w2df")
inverted_text.df = dict_text_w2df

In [33]:
doc_title_pairs = parquetFile.select("title", "id").rdd

In [34]:
#rdd of (word, (doc_id, df, doc_size))
title_word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

In [52]:
# creates an rdd of (word, [id,id,id])
title_word_id_list_rdd = title_word_counts.map(lambda x: (x[0], [x[1][0]])).reduceByKey(lambda x, y: x + y)

In [36]:
stemmed_title_word_id_dict = title_word_id_list_rdd.collectAsMap()

In [41]:
def not_stemmed_word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, f, doc size)) pairs
      for example: [("Anarchism", (12, 5, 35)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  ##contains the words and how many times it appeared in the given document
  word_dict={}
  num_of_words_per_doc = 0
  for w in tokens:
    if w not in all_stopwords:
        num_of_words_per_doc = num_of_words_per_doc+1
        #if we didn't see this word
        if w not in word_dict:
            word_dict[w] = [id,1]
        else:
            word_dict[w][1] = word_dict[word][1]+1
  return [(word, (tup[0],tup[1],num_of_words_per_doc)) for word,tup in word_dict.items()]

In [42]:
#creates an rdd of (word, (id, tf, id size))
not_stemmed_title_word_counts = doc_text_pairs.flatMap(lambda x: not_stemmed_word_count(x[0], x[1]))

In [44]:
not_stemmed_title_word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# creates an rdd of (word, [id,id,id])
not_stemmed_title_word_id_list_rdd = not_stemmed_title_word_counts.map(lambda x: (x[0], [x[1][0]])).reduceByKey(lambda x, y: x + y)
#creates a dict from the rdd
not_stemmed_title_word_id_dict = not_stemmed_title_word_id_list_rdd.collectAsMap()

In [45]:
# save_dict_to_pickle(not_stemmed_title_word_id_dict, "not_stemmed_title_id_dict")

In [ ]:
# write the global stats out
inverted_text.write_index('postings_gcp', 'text_index_update', bucket_name)
# upload to gs
index_src = "text_index_update.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [53]:
id_title_dict = load_dict_from_pickle_bucket("18071996","files/id_title_dict")

In [ ]:
# save_dict_to_pickle(inverted_text, "inverted_text")

In [ ]:
!gsutil ls -lh $index_dst